In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
from functools import partial
from dask.local import get_async
from dask import bag as db
from dask import array as da
import sprite2
import numpy as np
from matplotlib import pyplot as plt
import sprite2.aws
import sprite2.dask
import os
import math
import dask

In [24]:
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)-5s %(asctime)s %(threadName)s: %(message)s')
logging.debug("test1")

root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)

logger = logging.getLogger('sprite2')
logger.setLevel(logging.INFO)

In [27]:
def is_prime(n):
    if n % 2 == 0 and n > 2: 
        return False
    return all(n % i for i in range(3, int(math.sqrt(n)) + 1, 2))

def find_primes(rng):
    for i in rng:
        if is_prime(i):
            yield i

In [18]:
# create 100 ranges of 100,000 numbers
start = 0
stop = 10000000
size = 100000
ranges = [range(i,i+size-1) for i in range(start, stop, size)]
len(ranges), ranges[:10],ranges[-10:]

(100,
 [range(0, 99999),
  range(100000, 199999),
  range(200000, 299999),
  range(300000, 399999),
  range(400000, 499999),
  range(500000, 599999),
  range(600000, 699999),
  range(700000, 799999),
  range(800000, 899999),
  range(900000, 999999)],
 [range(9000000, 9099999),
  range(9100000, 9199999),
  range(9200000, 9299999),
  range(9300000, 9399999),
  range(9400000, 9499999),
  range(9500000, 9599999),
  range(9600000, 9699999),
  range(9700000, 9799999),
  range(9800000, 9899999),
  range(9900000, 9999999)])

In [19]:
# for each range, map over range with find_prime function, flatten results
d = (db.from_sequence(ranges, partition_size=1)
      .map(find_primes)
      .flatten())

In [21]:
# compute on local machine using multiprocessing (all cores)
%time primes = d.compute()

CPU times: user 238 ms, sys: 50.9 ms, total: 288 ms
Wall time: 41.6 s


In [25]:
# compute on lambda, takes significantly less time
%time primes = d.compute(get=sprite2.dask.get)

CPU times: user 1.65 s, sys: 218 ms, total: 1.87 s
Wall time: 3.79 s


In [16]:
len(primes), primes[:5], primes[-5:]

(216809, [0, 1, 2, 3, 5], [2999911, 2999921, 2999933, 2999951, 2999957])